In [3]:
import pandas as pd
import os

In [4]:
#Importing feature table from QIIME2 
basedir = '/mnt/c/MinION_16S_barcodes/BLASTn/BLASTn/full_BLAST/'
table_csv = os.path.join(basedir, '20190805_illumina_dada2.fasta.SILVA.besthit.outfmt6_1.tsv')
table_df = pd.read_csv(table_csv, sep= '\t')

In [6]:
#Number of rows with unique OTU ID
taxa_df['#OTU ID'].nunique()

6676

In [7]:
#Importing BLAST Illumina output
table1_csv = os.path.join(basedir, '20190805_illumina_dada2.fasta.SILVA.besthit.outfmt6')
table1_df = pd.read_csv(table1_csv, sep= '\t', names = ["Query Seq-id","Subject Seq-id","evalue","bitscore","length","pident","nident","subject-GI","Subject_accession","Subject_Taxonomy_ID","Start_of_alignment","End_of_alignment"])

In [9]:
#Number of unique ID present in both QIIME2 feature table and BLAST output
taxa_df['#OTU ID'].isin(taxa1_df['Query Seq-id']).value_counts()

True     6612
False      64
Name: #OTU ID, dtype: int64

In [10]:
#Processing the BLAST output table to only include Query seq-id and primaryAccession columns; Splitting the subject_accession into three columns in a new dataframe and merging only the first column with the table1_df dataframe
table1_df.drop(columns=['Subject Seq-id', 'evalue', 'bitscore', 'length', 'pident','nident','subject-GI','Subject_Taxonomy_ID','Start_of_alignment','End_of_alignment'], inplace=True)
table_df_1 = table1_df["Subject_accession"].str.split(".",n=2,expand = True)
table_df_1.columns = ['A', 'B', 'C']
result = pd.concat([table1_df, table_df_1], axis=1)
result.drop(columns=['Subject_accession', 'B','C'], inplace=True)
result.rename(columns={'A': 'primaryAccession'}, inplace=True)

In [12]:
#Number of unique accession numbers
result['primaryAccession'].nunique()

2105

In [13]:
#Merging the feature table from QIIME2 with the above processed table
merged_df_1 = pd.merge(table_df,result, how = 'left', left_on = '#OTU ID', right_on = 'Query Seq-id')
merged_df_1.fillna(value=0, inplace=True)

In [14]:
#dropping the unrequired columns
merged_df_1.drop(columns=['#OTU ID', 'Query Seq-id'], inplace=True)

In [16]:
#reordering coulumn
merged_df_2 = merged_df_1[['primaryAccession', 'Hare-control', 'MF-136', 'MF-138', 'MF-139','MF-140','MF-141','MF-142','MF-143','MF-144','MF-145','MF-146','MF-147','MF-148','MF-149','MF-150','MF-151','MF-152','MF-153','MF-154','MF-155','MF-156','MF-157','Rabbit-control','Stage1-NTC']]

In [18]:
# renaming the column
merged_df_2.rename(columns={'primaryAccession':'#OTU ID' }, inplace=True)

In [19]:
#grouping all the samples with their OTU ID by taking sum 
merged_df_3=merged_df_2.groupby(['#OTU ID']).sum()

In [26]:
#importing silva_132 taxonomy mapping file
basedir2 = '/mnt/c/MinION_16S_barcodes/BLASTn/BLASTn/taxonomy_files/'
map_csv = os.path.join(basedir2, 'taxmap_slv_ssu_ref_132-corrected.txt')
map_df = pd.read_csv(map_csv, sep= '\t')

In [27]:
#dropping columns
map_df.drop(columns=['start','stop','organismName','taxid'], inplace=True)

In [29]:
#removing duplicates
map1_df = map_df.drop_duplicates(subset='primaryAccession', keep='first', inplace=False)

In [31]:
#merging the processed feature table with the mapping file
merged_df_4 = pd.merge(merged_df_3,map1_df, how = 'left', left_on = '#OTU ID', right_on = 'primaryAccession')
merged_df_4.fillna(value='unassigned', inplace=True)

In [33]:
#dropping the columns not required for taxonomy file to use within QIIME2
merged_df_4.drop(columns=['primaryAccession','Hare-control', 'MF-136', 'MF-138', 'MF-139','MF-140','MF-141','MF-142','MF-143','MF-144','MF-145','MF-146','MF-147','MF-148','MF-149','MF-150','MF-151','MF-152','MF-153','MF-154','MF-155','MF-156','MF-157','Rabbit-control','Stage1-NTC'],inplace=True)

In [35]:
#renaming columns
merged_df_4.rename(columns={'#OTU ID': 'Feature ID'}, inplace=True)

In [36]:
merged_df_4.rename(columns={'path': 'Taxon'}, inplace=True)

In [38]:
#Taxonomy file
merged_df_4.to_csv(os.path.join(basedir1, '20190821_taxamap_illumina_dada2.tsv'),sep='\t',header=True,index =False)

In [ ]:
#convert the feature table to biom format and import into qiime2
#Taxonomy file can be imported in tsv format